# 模型构建

这里主要阐述模型构建中采用的数据和构造的数据，关于模型的具体内容参考：《如何通过LambdaMart算法预测未来收益最佳基金》

考虑到模型的一般性、泛化性，同时考虑算力和实用性，这里采用每周采用。即，每周日回测基金过往历史数据，同时测算其在接下来一段时间的收益率，意在构建通过历史数据预测基金未来收益排名算法。

数据主要包括：

1. 量化指标：

    1.1 量化指标从时间维度：过去5年，过去3年，过去2年，过去1年，过去半年，
    过去一季度，过去一月，过去一周；
    
    1.2 量化指标从指标维度：收益率、年化收益率、最大回测率、年化波动率、
    alpha、beta，SharpeRatio，TreynorRatio，JensenAlpha，DownsideDeviation，
    SortinoRatio，TrackError，InformationRatio；
 
 
2. 基础数据：

    2.1 基金公司

    2.2 基金规模：基金最近规模数据，近一年分位数，基金总规模、总规模近5年分位数，
    同类总规模、各类基金规模近5年分位数，各类基金规模占总规模比例
    
    2.3 基金分红：累计分红和近一年累计分红
    
    2.4 市场指标：上证指数点位、上证指数估值（PE-TTM）、十年期国债利率、
    1/估值-国债利率、以及它们各自的最近5年分位数；


3. 筛选：

    3.1  数据采用2005年之后的数据
    
    3.2 成立满一年
    
    3.3 剔除过去5年和过去一周的量化指标，缺失太多
    
    3.4 选择普通股票型和偏股混合型
    

# 选基模型结果分析报告

In [ ]:
import pandas as pd
import numpy as np
from pylab import mpl
import matplotlib.pyplot as plt
import seaborn as sns
#sns.set(style="white", rc={"axes.facecolor": (0, 0, 0, 0)})
import pymysql
from xgboost import plot_tree
import pprint
from IPython.display import display
import warnings
warnings.simplefilter(action='ignore', category=UserWarning)

#pd.set_option('display.height',1000)
#pd.set_option('display.max_rows',500)
#pd.set_option('display.max_columns',500)
pd.set_option('display.width',100)

In [ ]:
Next1month = pd.read_excel('../new_training/Next1month_infomation.xlsx', header=[0], index_col=0)
Next1quar = pd.read_excel('../new_training/Next1quar_infomation.xlsx', header=[0], index_col=0)
Next6month = pd.read_excel('../new_training/Next6month_infomation.xlsx', header=[0], index_col=0)
Next1year = pd.read_excel('../new_training/Next1year_infomation.xlsx', header=[0], index_col=0)

In [ ]:
class Predict_Analysis():
    def __init__(self, analysis_df):
        colnames = analysis_df.columns
        colnames = [(colname[colname.find('(')+2:colname.find(',')-1], colname[colname.find(',')+3:colname.find(')')-1]) if colname[0]=='(' else colname for colname in colnames ]
        analysis_df.columns = colnames
        self.analysis_df = analysis_df

    def _seri_base(self, x):
        x = x[x.notnull()]
        return pd.Series([x.min(), x.quantile(0.25), x.median(), x.quantile(0.75), x.max(), x.mean(), len(x[x>=0])/len(x)*100], \
                         index=['min', '25%', '50%', '75%', 'max', 'mean', 'pos_rate']).round(2)

    def _win_rate1(self, x, y):
        x = np.array(x)
        y = np.array(y)
        z = y-x
        return round(len(z[z>=0])/len(x)*100,2)
    
    def _win_rate2(self, x, y):
        rate = []
        for col in y.columns:
            x = np.array(x)
            z = np.array(y[col])
            m = z-x
            rate.append(round(len(m[m>=0])/len(m)*100,2))
        return pd.Series(rate, y.columns)
        
    def print_infomation(self):
        print('##################################################################################################################')
        print('-------------------- 数据集的包含如下数据：----------------------------')
        #print(self.analysis_df.columns)
        display(self.analysis_df.columns)
        print('##################################################################################################################')
        print('总共回测历史数据')
        print('从', self.analysis_df.index[0])
        print('至', self.analysis_df.index[-1])
        print('总共在%s周，每周末对未来基金收益率进行回测建模，并对结果进行分析'%len(self.analysis_df))

    def print_allfund_infomation(self):
        self.info_all = self.analysis_df.loc[:, ['count', 'min', '25%', '50%', '75%', 'max', 'mean', 'pos_rate',]]
        print('##################################################################################################################')
        print('--------------------- 每个时间点基金收益分布如下： ----------------------------------------------')
        #pprint.pprint(self.info_all.head(10))
        display(self.info_all.head(10))
        self.allinfo_desc = self.info_all.iloc[:, 2:].apply(self._seri_base)
        print('##################################################################################################################')
        print('--------------------- 全部基金收益对所有分析时间统计数据如下： ----------------------------------------------')
        #pprint.pprint(self.allinfo_desc)
        display(self.allinfo_desc)
    
    
    def _density_plot1(self, metric, quan_min=0, quan_max=1):
        if int(metric[-1]) == 1:
            analysis_sub = self.analysis_df.loc[:,metric]
        else:
            analysis_sub = self.analysis_df.loc[:,[(metric, 'mean'),(metric, 'median')]]
        plot_df = pd.concat([analysis_sub, self.info_all[['25%', '50%', '75%', 'mean']]], axis=1)
        df = plot_df.stack().reset_index(1)
        df.columns = ['class','return']
        
        sns.set(style="white", rc={"axes.facecolor": (0, 0, 0, 0)})
        
        #my_dpi = 96
        #plt.figure(figsize=(2000/my_dpi, 1000/my_dpi), dpi=my_dpi)
        
        pal = sns.cubehelix_palette(10, rot=-.25, light=.7)
        g = sns.FacetGrid(df, row="class", hue="class", aspect=10, size=1, palette=pal)
        
        g.map(sns.kdeplot, "return", clip_on=False, shade=True, alpha=1, lw=1.5, bw=.2)
        g.map(sns.kdeplot, "return", clip_on=False, color="w", lw=2, bw=.2)
        g.map(plt.axhline, y=0, lw=2, clip_on=False)
        
        def label(x, color, label):
            ax = plt.gca()
            ax.text(0, .2, label, fontweight="bold", color=color,
                    ha="left", va="center", transform=ax.transAxes)
        
        g.map(label, "return")
        
        g.fig.subplots_adjust(hspace=-.25)
        
        g.set_titles("")
        g.set(yticks=[])
        g.set(xlim=(np.quantile(df['return'], quan_min), np.quantile(df['return'], quan_max)))
        g.despine(bottom=True, left=True)
        plt.show()

    
    def _timeseries_plot1(self, metric):
        if int(metric[-1]) == 1:
            analysis_sub = self.analysis_df.loc[:,metric]
        else:
            analysis_sub = self.analysis_df.loc[:,[(metric, 'mean')]]
        plot_df = pd.concat([analysis_sub, self.info_all[['25%','50%', '75%']]], axis=1)
        #plot_df = pd.concat([analysis_sub, self.info_all[['25%', '50%', '75%', 'mean']]], axis=1)
        my_dpi = 96
        fig, axs = plt.subplots(nrows=1, ncols=1, figsize=(1200/my_dpi, 500/my_dpi), dpi=my_dpi)
        plot_df.plot(ax=axs, legend=False)
        plt.legend(facecolor='white')
        plt.show()

    def _timeseries_plot2(self, metric):
        if int(metric[-1]) > 1:
            analysis_sub = self.analysis_df.loc[:,[(metric, 'pos_rate')]]
            plot_df = pd.concat([analysis_sub, self.info_all[['pos_rate']]], axis=1)
            my_dpi = 96
            fig, axs = plt.subplots(nrows=1, ncols=1, figsize=(1000/my_dpi, 350/my_dpi), dpi=my_dpi)
            plot_df.plot(ax=axs, legend=False)
            plt.legend(loc='center left', bbox_to_anchor=(1.01, 0.5), facecolor='white')
            plt.show()
    
    def metric_info(self, metric, quan_min=0, quan_max=1):
        if int(metric[-1]) == 1:
            analysis_sub = self.analysis_df.loc[:,metric]
            info_sub = self._seri_base(analysis_sub).rename(metric)
            concat_all = pd.concat([info_sub, self.allinfo_desc], axis=1)
            win_rate = self.info_all.iloc[:,1:7].apply(lambda x: self._win_rate1(x, analysis_sub), axis=0)
        else:
            analysis_sub = self.analysis_df.loc[:,[(metric,'mean'),(metric,'median')]]
            info_sub = analysis_sub.apply(self._seri_base, axis=0)
            concat_all = pd.concat([info_sub, self.allinfo_desc[['25%','50%','75%','mean']]], axis=1)
            win_rate = self.info_all.iloc[:,1:].apply(lambda x: self._win_rate2(x, analysis_sub), axis=0)
        print('-----------选中基金收益与全部基金收益指标比较-----------')
        #pprint.pprint(concat_all)
        display(concat_all)
        print('-----------选中基金收益与全部基金收益指标比较图-----------')
        self._density_plot1(metric, quan_min, quan_max)
        self._timeseries_plot1(metric)
        if int(metric[-1]) > 1:
            self._timeseries_plot2(metric)
        print('-----------选中基金收益与全部基金收益指标同时间战胜比例-----------')
        #pprint.pprint(win_rate)
        display(win_rate)
    
    
    def _find_why(self, all_index, select_index):
        
        analysis_sub1 = self.analysis_df.loc[:,[select_index in col for col in self.analysis_df.columns]]
        if all_index == select_index:
            analysis_sub2 = self.analysis_df[['ndcg@1']]
        else:
            analysis_sub2 = self.analysis_df[[all_index,'ndcg@1']]
        analysis_sub = pd.concat([analysis_sub2, analysis_sub1], axis=1)
        del analysis_sub1, analysis_sub2
        predict = analysis_sub.drop(columns=all_index, axis=1)
        real = analysis_sub[all_index]
        
        def reduce(x):
            return x-real
        
        reduce_df = predict.apply(reduce, axis=0)
        
        def pos_rate(x):
            return len(x[x>=0])
        
        pos_num = reduce_df.apply(pos_rate, axis=1)
        pos_num = pos_num.rename('%s_%s'%(all_index, select_index))
        return pos_num
    
    def _find_why_fut(self):
        pos = []
        for all_index in ['50%','mean']:
            for select_index in ['mean','median']:
                pos.append(self._find_why(all_index , select_index))
        pos.append(self.analysis_df[['25%','50%','75%','mean']])
        pos = pd.concat(pos, axis=1)
        
        db = pymysql.connect(host = "118.31.72.134", port = 3306, user = "taoji", passwd = "Abc12345", db = "dzh", charset = 'utf8')
        cursor2 = db.cursor()
        sql2 = "SELECT t.TRADEDATE,t.TCLOSE FROM TQ_QT_INDEX t LEFT JOIN TQ_IX_BASICINFO t1 ON t.SECODE=t1.SECODE WHERE t1.SYMBOL='000001' ORDER BY t.TRADEDATE"
        cursor2.execute(sql2)
        szzz_data = pd.DataFrame(list(cursor2.fetchall()),columns=['date','szzz'])
        szzz_data = szzz_data.astype({'date':'datetime64','szzz':'float'})
        db.close()
        szzz_data = szzz_data.set_index('date')
        
        pos = pd.concat([pos, szzz_data], sort=True)
        pos = pos.sort_index(ascending=True)
        pos['szzz'] = pos['szzz'].fillna(method='ffill')
        pos = pos.dropna()
        self.pos = pos
    
    def analysis_plot(self):
        self._find_why_fut()
        pos3 = self.pos['szzz']
        pos2 = self.pos[['25%','50%','75%','mean']]
        pos1 = self.pos[['50%_mean','mean_mean','50%_median','mean_median']]
        #mpl.rcParams['font.sans-serif'] = ['FangSong']
        plt.rcParams['font.sans-serif'] = ['Arial Unicode MS']
        mpl.rcParams['axes.unicode_minus'] = False
        my_dpi = 96
        fig, axs = plt.subplots(nrows=3, ncols=1, figsize=(1000/my_dpi, 800/my_dpi), dpi=my_dpi)
        plt1 = pos1.plot(ax=axs[0])
        plt1.set_title('各选择平均战胜全部平均数')
        plt1.legend(loc='center left', bbox_to_anchor=(1.01, 0.5),facecolor='white')
        axs[0].set_xticks([])
        plt2 = pos2.plot(ax=axs[1])
        plt2.set_title('权益类基金未来收益分布')
        plt2.legend(loc='center left', bbox_to_anchor=(1.01, 0.5),facecolor='white')
        axs[1].set_xticks([])
        plt3 = pos3.plot(ax=axs[2])
        plt3.set_title('当时的上证综指')
        plt.show()

## 预测未来一月收益 Top N 基金

In [ ]:
N1m = Next1month.copy()
n1m = Predict_Analysis(N1m)
n1m.print_infomation()
n1m.print_allfund_infomation()

从上述数据可以发现：
1. 在2018年2月11日至2019年5月5日测算的65个时间点（65个周末），基金未来一月收益整体表现不佳。
2. 最差一周出现 25%的基金亏损超过12.55%， 一半基金亏损超过10%， 75%基金亏损超过8.8%，没有一个基金获得正收益，平均收益只有-10.87%；
3. 有5%的时间所有基金都为负收益；
4. 有超过一半的时间（33周）将近75%的基金都未获得正收益，平均收益未-1.34%；
5. 当然也存在收益较好的时间，50%以上基金获得13.26%以上收益，平均收益14.23%，而最优基金获得42.38%；

### 选择一个

In [ ]:
n1m.metric_info('ndcg@1')

从上述图标比较可以发现：
1. 从总体而言基金选择的基金的收益与全部基金比较战胜中位数、均值，大部分指标不弱于上四分位数；
2. 选择的基金除了正收益比例和上四分位数外，其它指标全部优于各个时间收益的中位数和均值的分布；
3. 选择的基金与各个时间收益的上四分位数的分布比较，下四分数有优势而上四分位数、最大值和正收益比例却不如；
4. 对应时间比较看，只有50%左右时间能够战胜均值和中位数，而胜上四分位数的时间只有27%。

原因：选择的基金在市场整体表现较差的时候表现较为优异，但是在市场表现极为出色的时候表现也较为一般。

### 选择两个

In [ ]:
n1m.metric_info('ndcg@2')

从上述图标比较可以发现：
1. 从总体而言选择基金的收益与全部基金比较战胜中位数、均值；
2. 选择的基金在25%分位数、中位数、均值和最大值都战胜各时间中位数、均值对应的指标，其它指标也都不弱于中位数、均值对应指标；
3. 25%分位数和最值上甚至战胜每个时间点的上四分位数
4. 对应时间比较看，只有60%左右时间能够战胜中位数，52.31%时间战胜均值，而胜上四分位数的时间只有26.15%。

原因：有一些时间点基金收益存在分化。

### 选择三个

In [ ]:
n1m.metric_info('ndcg@3')

从上述图标比较可以发现：
1. 从总体而言选择基金的收益与全部基金比较全面战胜中位数和均值；
2. 选择的基金组合所有指标都战胜各个时间点全部基金的中位数和均值；
3. 选择的基金组合最差的情况与75%分位数最差情况差不多，而25%分位数和最大值优于75%分位数
4. 对应时间比较看，有64.62%时间能够战胜中位数，61.54%时间战胜均值，而胜上四分位数的时间将近30%，从未弱于25%分位数。

原因：选择的基金组合能够收益分布更加集中。

### 选择五个

In [ ]:
n1m.metric_info('ndcg@5')

从上述图标比较可以发现：
1. 从总体而言选择基金的收益与全部基金比较全面战胜中位数和均值；
2. 选择的基金组合几乎所有指标都战胜各个时间点全部基金的中位数和均值；
3. 选择的基金组合除50%、75%和max，不弱于75%分位数；
4. 对应时间比较看，有70.77%时间能够战胜中位数，70.77%时间战胜均值，而胜上四分位数的时间只要27.69%，从未弱于25%分位数。

原因：基金在一些时刻出现了分化（部分基金收益特别低），在整体表现较差时刻选择的组合能够不弱于上四分位数。

### 原因

In [ ]:
n1m.analysis_plot()

从时间维度看，预测的基金整体表现不佳的时刻主要集中在了基金整体表现较为优异时刻，特别是一些拐点时刻。

## 预测未来一季度收益 Top N 基金

In [ ]:
N1q = Next1quar.copy()
n1q = Predict_Analysis(N1q)
n1q.print_infomation()
n1q.print_allfund_infomation()

从上述数据可以发现：
1. 在2017年12月17日至2019年3月3日测算的64个时间点，基金未来一季度收益整体表现不佳。
2. 最差一季出现 25%的基金亏损超过20.82%， 一半基金亏损超过17%， 75%基金亏损超过-14.85%，没有一个基金获得正收益，平均收益为-17.84%；
3. 有25%的时间只有不到2.5%的基金获得正收益；
4. 有超过一半的时间（33周）将近75%的基金都未获得正收益，平均收益未-4.56%；
5. 当然也存在收益较好的时间，50%以上基金获得近33%收益，平均收益14.23%，而最优基金获得58%；

### 选择一个

In [ ]:
n1q.metric_info('ndcg@1')

从上述图标比较可以发现：
1. 从总体而言选择基金的收益与全部基金比较总体不如中位数和均值；
2. 选择的基金组合只战胜各个时间点全部基金的中位数和上四分位数；
4. 对应时间比较看，有54.69%时间能够战胜中位数，39%时间战胜均值，而胜上四分位数的时间只要9.38%，87.50%胜于25%分位数。

整体表现不佳。

### 选择两个

In [ ]:
n1q.metric_info('ndcg@2')

从上述图标比较可以发现：
1. 从总体而言选择基金的收益与全部基金比较总体优于中位数和均值；
2. 选择的基金组合在25%分位数、中位数和75分位数（即：主体部分），战胜各个时间点全部基金的中位数和均值；
4. 对应时间比较看，有40.62%时间能够战胜中位数，42.19%时间战胜均值，而胜上四分位数的时间只要14.06%，79.69%胜于25%分位数。

整体表现不佳。

### 选择三个

In [ ]:
n1q.metric_info('ndcg@3')

从上述图标比较可以发现：
1. 从总体而言选择基金的收益与全部基金比较总体优于中位数和均值；
2. 选择的基金组合在25%分位数、中位数和75分位数（即：主体部分），战胜各个时间点全部基金的中位数和均值；
4. 对应时间比较看，有48.44%时间能够战胜中位数，42.19%时间战胜均值，而胜上四分位数的时间只要15.62%，87.50%胜于25%分位数。

整体表现不佳。

### 选择五个

In [ ]:
n1q.metric_info('ndcg@5')

从上述图标比较可以发现：
1. 从总体而言选择基金的收益与全部基金比较总体优于中位数和均值；
2. 选择的基金组合在25%分位数、中位数和75分位数（即：主体部分），战胜各个时间点全部基金的中位数和均值；
4. 对应时间比较看，有50%时间能够战胜中位数，42.19%时间战胜均值，而胜上四分位数的时间只要4.69%，82.81%胜于25%分位数。

整体表现不佳。

### 原因

In [ ]:
n1q.analysis_plot()

对于未来一季度基金收益的预测，总体表现较一般。选择的基金组合虽然从总体上讲整体是略微优于中位数的分布的，但是优势不明显且受极端情况的影响较大。

## 预测未来半年收益 Top N 基金

In [ ]:
N6m = Next6month.copy()
n6m = Predict_Analysis(N6m)
n6m.print_infomation()
n6m.print_allfund_infomation()

从上述数据可以发现：
1. 在2017年10月1日至2018年12月2日测算的62个时间点，基金未来半年收益整体表现不佳。
2. 最差半年出现 25%的基金亏损超过25.55%，一半基金亏损超过21%，75%基金亏损超过17%，没有一个基金获得正收益，平均收益只有-21%；
3. 有近25%的时间基本所有基金都为负收益；
4. 有超过一半的时间超过75%的基金都未获得正收益，平均收益未不到1%；
5. 当然也存在收益较好的时间，50%以上基金获得28.51%以上收益，平均收益29.64%，而最优基金获得68.11%；

### 选择一个

In [ ]:
n6m.metric_info('ndcg@1')

从上述图标比较可以发现：
1. 从总体而言选择基金的收益与全部基金比较总体优于中位数和均值；
2. 选择的基金组合全面战胜各个时间点全部基金的中位数，但不及上四分位；
3. 选择的基金组合除最小值外战胜各个时间点全部基金的均值；
4. 对应时间比较看，有64.52%时间能够战胜中位数，56.45%时间战胜均值，而胜上四分位数的时间只要17.74%，88.71%胜于25%分位数。

### 选择两个

In [ ]:
n6m.metric_info('ndcg@2')

从上述图标比较可以发现：
1. 从总体而言选择基金的收益与全部基金比较总体优于中位数和均值；
2. 选择的基金组合除最小值和25%外，战胜各个时间点全部基金的中位数和均值；
3. 选择的基金组合完全不如75%；
4. 对应时间比较看，只有38.71%时间能够战胜中位数，40.32%时间战胜均值，而胜上四分位数的时间只要16.13%，79.03%胜于25%分位数。

### 选择三个

In [ ]:
n6m.metric_info('ndcg@3')

从上述图标比较可以发现：
1. 从总体而言选择基金的收益与全部基金比较总体不差于中位数和均值；
2. 选择的基金组合的50%、75%、max，正收益比例指标战胜各个时间点全部基金的中位数，但除75%外不及上四分位；
3. 选择的基金组合的50%、75%、max指标战胜各个时间点全部基金的均值；
4. 对应时间比较看，有51.61%时间能够战胜中位数，50%时间战胜均值，而胜上四分位数的时间只要6.45%，83.87%胜于25%分位数。

原因：整体表现差的时候选择的组合也表现得更差。

### 选择五个

In [ ]:
n6m.metric_info('ndcg@5')

从上述图标比较可以发现：
1. 从总体而言选择基金的收益与全部基金比较总体优于中位数和均值，逼近上四分位数；
2. 选择的基金组合的各个指标战胜各个时间点全部基金的中位数和均值，且逼近上四分位；
3. 对应时间比较看，有66.13%时间能够战胜中位数，61%时间战胜均值，而胜上四分位数的时间只要16.13%，95.16%胜于25%分位数。

原因：模型表现可以，整体表现差的时候选择的组合也表现得差。

### 原因

In [ ]:
n6m.analysis_plot()

## 预测未来一年收益 Top N 基金

In [ ]:
N1y = Next1year.copy()
n1y = Predict_Analysis(N1y)
n1y.print_infomation()
n1y.print_allfund_infomation()

从上述数据可以发现：
1. 在2017年4月16日至2018年6月3日测算的60个时间点，基金未来一年收益整体表现不佳。
2. 最差一年出现25%的基金亏损超过30%，一半基金亏损超过26%，75%基金亏损超过22%，没有一个基金获得正收益，平均收益只有-26.56%；
3. 有近25%的时间所有基金都为负收益；
4. 有超过一半的时间超过75%的基金都未获得正收益，平均收益未-7.71%；
5. 当然也存在收益较好的时间，50%以上基金获得13%以上收益，平均收益13.42%，而最优基金获得51.39%；

### 选择一个

In [ ]:
n1y.metric_info('ndcg@1')

从上述图标比较可以发现：
1. 从总体而言选择基金的收益与全部基金比较总体优于中位数和均值，不如75%分位数；
2. 对应时间比较看，只有有38.33%时间能够战胜中位数，36.67%时间战胜均值，而胜上四分位数的时间只要16.67%，80%得时间胜于25%分位数。

### 选择两个

In [ ]:
n1y.metric_info('ndcg@2')

从上述图标比较可以发现：
1. 从总体而言选择基金的收益与全部基金比较总体完全优于中位数和均值；
2. 对应时间比较看，有76.67%时间能够战胜中位数，73.33%时间战胜均值，而胜上四分位数的时间只要26.67%，全部时间胜于25%分位数。

原因：模型表现出色，特别是在整体表现差的时刻表现出色。

### 选择三个

In [ ]:
n1y.metric_info('ndcg@3')

从上述图标比较可以发现：
1. 从总体而言选择基金的收益与全部基金比较总体不及中位数和均值；
2. 对应时间比较看，有41.67%时间能够战胜中位数，41.67%时间战胜均值，而胜上四分位数的时间只要6.367%，71.67%得时间胜于25%分位数。

原因：模型表现较为一般。

### 选择五个

In [ ]:
n1y.metric_info('ndcg@5')

从上述图标比较可以发现：
1. 从总体而言选择基金的收益与全部基金比较总体完全优于中位数和均值；
2. 对应时间比较看，有95%时间能够战胜中位数，93.33%时间战胜均值，而胜上四分位数的时间只要26.67%，全部时间胜于25%分位数。

原因：模型表现出色，特别是在整体表现差的时刻表现出色。

### 原因

In [ ]:
n1y.analysis_plot()

对未来一年基金收益预测排序模型整体表现出色，特别是采用map度量下的结果。

## 结论

1. 对未来一月基金收益预测排序模型整体表现可以，而对未来一季度和半年基金收益预测排序模型整体表现不佳，但是对于未来一年基金收益模型表现出色，特别是采用map度量后。

2. 对于模型的改进，还需要能够发现针对各模型缺陷的特征。

3. 在2的基础上还可以尝试探索通过深度神经网络发现一些深度非线性关系。